In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import pickle
from app.util.timer import Timer
from app.util.Differ import Differ
from main import YoloRuntimeTest
from functools import partial

Check CUDA available

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()

GPU input

In [2]:
args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_3.jpg", 
    "classes": "./app/weights/metadata.yaml", 
    "type": "image",
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cuda:0"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_3.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cuda:0"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_3.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cuda:0"
}

Initilize YOLO runtime test class

In [3]:
yolo_runtime_test = YoloRuntimeTest()

In [5]:
gpu_pytorch_image = yolo_runtime_test.ultralytics_run_image(args_pytorch)
print(gpu_pytorch_image)

[INFO] Initialize Model
[INFO] Inference Image


session run: 0.008196900000001506
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_3.jpg: 480x640 1 person, 4 cars, 6 traffic lights, 8.0ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_3.jpg: 480x640 1 person, 4 cars, 6 traffic lights, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Elapsed time: 0.2345 seconds
Class: car, Confidence: 0.93, Box: [1, 2018, 664, 2355]
Class: car, Confidence: 0.91, Box: [815, 2097, 1315, 2295]
Class: traffic light, Confidence: 0.82, Box: [1346, 1263, 1408, 1428]
Class: traffic light, Confidence: 0.82, Box: [1907, 1279, 1966, 1447]
Class: person, Confidence: 0.82, Box: [1487, 2043, 1569, 2324]
Class: traffic light, Confidence: 0.81, Box: [1162, 1202, 1228, 1366]
Class: traffic

In [ ]:
gpu_openvino_image = yolo_runtime_test.ultralytics_run_image(args_openvino)
print(gpu_openvino_image)

In [4]:
gpu_onnx_image = yolo_runtime_test.ultralytics_run_image(args_onnx)
print(gpu_onnx_image)

[INFO] Initialize Model
[INFO] Inference Image
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...

session run: 0.01912209999999881
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_3.jpg: 640x640 1 person, 4 cars, 6 traffic lights, 22.7ms
Speed: 38.6ms preprocess, 22.7ms inference, 628.6ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 9.8181 seconds
Class: car, Confidence: 0.93, Box: [1, 2018, 663, 2354]
Class: car, Confidence: 0.91, Box: [815, 2097, 1315, 2295]
Class: person, Confidence: 0.82, Box: [1487, 2043, 1569, 2323]
Class: traffic light, Confidence: 0.82, Box: [1346, 1262, 1409, 1428]
Class: traffic light, Confidence: 0.81, Box: [1907, 1279, 1967, 1447]
Class: traffic light, Confidence: 0.81, Box: [1162, 1202, 1228, 1366]
Class: traffic light, Confidence: 0.80, Box: [3014, 773, 3155, 1041]
Class: traffic light, Confidence: 0.76, Box: [629, 1187, 698, 1353]
Class: car, Confidence: 0.71, Box: [1799, 2216, 1914, 

In [ ]:
gpu_onnx_runtime_image = yolo_runtime_test.onnxruntime_run_image(args_onnx)
print(gpu_onnx_runtime_image)

Difference GPU

In [ ]:
pd.set_option('display.expand_frame_repr', False)

def generate_difference_df(image1, image2, label):
    differ = Differ(np.array(image1), np.array(image2))
    result = differ.find_difference()
    return pd.DataFrame(result, columns=[label, "gpu conf_diff", "gpu box_diff (px)"])

df_pt_openvino = generate_difference_df(gpu_pytorch_image[0], gpu_openvino_image[0], "pt vs openvino+ultralytics")
df_pt_onnx = generate_difference_df(gpu_pytorch_image[0], gpu_onnx_image[0], "pt vs onnx+ultralytics")
df_pt_onnxruntime = generate_difference_df(gpu_pytorch_image[0], gpu_onnx_runtime_image[0], "pt vs onnxruntime")

df_combined = pd.concat([df_pt_openvino, df_pt_onnx, df_pt_onnxruntime], axis=1)

print(df_combined)

Average GPU Time (10)

In [ ]:
result_time = []

In [ ]:
def collect_execution_times(run_inference_func, iterations=10):
    execution_times = []
    for _ in range(iterations):
        execution_time = run_inference_func()
        execution_times.append(execution_time[1] * 100)
    return execution_times

pytorch_func = partial(yolo_runtime_test.ultralytics_run_image, args=args_pytorch)
openvino_func = partial(yolo_runtime_test.ultralytics_run_image, args=args_openvino)
onnx_func = partial(yolo_runtime_test.ultralytics_run_image, args=args_onnx)
onnx_runtime_func = partial(yolo_runtime_test.onnxruntime_run_image, args=args_onnx)

result_time.append(collect_execution_times(pytorch_func))
result_time.append(collect_execution_times(openvino_func))
result_time.append(collect_execution_times(onnx_func))
result_time.append(collect_execution_times(onnx_runtime_func))

In [ ]:
result_time = np.array(result_time)
df = pd.DataFrame(np.transpose(result_time), 
                  columns=["pytorch time gpu (ms)",
                           "openvino+ultralytics time gpu (ms)",
                           "onnx​+ultralytics time gpu (ms)", 
                           "onnx runtime time gpu (ms)"])
df.describe(percentiles=[.9, .95])

Save GPU result

In [ ]:
with open('./app/saved_pkl/gpu_df.pkl', 'wb') as f:
    pickle.dump(df, f)